In [2]:
from pathlib import Path
import pandas as pd
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [3]:
output_dir = Path(r'C:\Users\luke1\Desktop\projects\T-brain-SinoPac-AIGO\data\public_dataset.csv')
data_columns = {
    'ID': 'ID',
    '縣市': 'City',
    '鄉鎮市區': 'District',
    '路名': 'Street_Name',
    '土地面積': 'Land_Area',
    '使用分區': 'Zoning',
    '移轉層次': 'Transfer_Level',
    '總樓層數': 'Total_Floors',
    '主要用途': 'Primary_Use',
    '主要建材': 'Primary_Construction_Material',
    '建物型態': 'Building_Type',
    '屋齡': 'Age_of_Building',
    '建物面積': 'Building_Area',
    '車位面積': 'Parking_Area',
    '車位個數': 'Number_of_Parking_Spaces',
    '橫坐標': 'Longitude',
    '縱坐標': 'Latitude',
    '備註': 'Remarks',
    '主建物面積': 'Main_Building_Area',
    '陽台面積': 'Balcony_Area',
    '附屬建物面積': 'Auxiliary_Building_Area',
    '單價': 'Unit_Price'
    }


In [5]:
def main():
    # asyncio.run(main_async())

    df = pd.read_csv(output_dir)

    df = df.rename(columns=data_columns)

    df_result = df['ID']
    df=df.drop(["ID", "Longitude","Latitude", "Street_Name", "Remarks"], axis=1)
    df['Address'] = df['City'] + df['District']
    df = df.drop(columns=['City', 'District'])

    # One hot encoding example
    # df = pd.get_dummies(df, columns=['Primary_Use'])

    # Label encoding
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df['Zoning'] = label_encoder.fit_transform(df['Zoning'])
    Zoning_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Primary_Use'] = label_encoder.fit_transform(df['Primary_Use'])
    Primary_Use_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Primary_Construction_Material'] = label_encoder.fit_transform(df['Primary_Construction_Material'])
    Primary_Construction_Material_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Building_Type'] = label_encoder.fit_transform(df['Building_Type'])
    Building_Type_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Address'] = label_encoder.fit_transform(df['Address'])
    Address_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))


    # Feature Scaling
    scaler = StandardScaler()
    df['Land_Area'] = scaler.fit_transform(df['Land_Area'].values.reshape(-1, 1))
    df['Age_of_Building'] = scaler.fit_transform(df['Age_of_Building'].values.reshape(-1, 1))
    df['Building_Area'] = scaler.fit_transform(df['Building_Area'].values.reshape(-1, 1))
    df['Parking_Area'] = scaler.fit_transform(df['Parking_Area'].values.reshape(-1, 1))
    df['Main_Building_Area'] = scaler.fit_transform(df['Main_Building_Area'].values.reshape(-1, 1))
    df['Balcony_Area'] = scaler.fit_transform(df['Balcony_Area'].values.reshape(-1, 1))
    df['Auxiliary_Building_Area'] = scaler.fit_transform(df['Auxiliary_Building_Area'].values.reshape(-1, 1))
    # df['Unit_Price'] = scaler.fit_transform(df['Unit_Price'].values.reshape(-1, 1))

    model = Sequential()
    model = pickle.load(open(r'output\output.pickle', "rb"))
    predictions = model.predict(df)
    predictions_column = pd.DataFrame(predictions, columns=['predicted_price'])
    df_result = pd.concat([df_result, predictions_column], axis=1)
    # print(df_result)
    df_result.to_csv(r'output/result.csv', index=False)

if __name__ == "__main__":
    main()